# 誤差逆伝播法(Backpropagation) と BPTT(Backpropagation Trough Time)


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation as ani
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Hiragino Maru Gothic Pro', 'Yu Gothic', 'Meirio', 'Takao', 'IPAexGothic', 'IPAPGothic', 'VL PGothic', 'Noto Sans CJK JP']

## 合成関数と連鎖律

関数 $y = f(x)$ と $z = g(y)$ を「**合成**」する。

すると、$f$ を適用した後に $g$ を適用するため、$z=g(f(x))$ になる。

ディープラーニングのニューラルネットワークでは、ニューロンを重ねて複雑な関数を表現する。各ニューロンを「１つの関数」と見ると、ニューラルネットワークは、多くの関数を合成した「**合成関数**」とみなせる。

ニューラルネットワークのような合成関数を簡単に微分する方法として「**連鎖律(chain rule)**」がある。

---

$(3x+4)^2$

という式を計算してみる。

そのまま展開して微分すると、

$=9x^2+24x+16$

$=18x+24$

2乗だと簡単だが、3乗や4乗と増えていくと式を展開するのが大変になっていく。そこで、合成関数の微分を使う。

公式として以下にまとめられる。

$u = (3x+4)^2$ とおく。$u=g(x)$になる。


$\frac{d}{dx} f(g(x)) = \frac{df(u)}{du}\frac{du}{dx}$


数値を入れてみる。

$(3x+4)^2$

$= \frac{d}{dx} \{(3x+4)^2\}$

$= \frac{du}{dx} \frac{d}{du}(u^2)$

$= \frac{d}{dx}(3x+4)*\frac{d}{du}(u^2)$

$=3*2u=6u$

$=6(3x+4)$

$=18x+24$

となり、展開して微分した結果と同じになる。

---

偏微分を連鎖率で解く

$\dfrac{\partial}{\partial x} (x - 10)^{2} + (x - y)^{2} + (y - 5)^{2}
\\ = 2(x-10)\times1 + 2(x-y)\times 1
\\ = 4x -2y -20$

$\dfrac{\partial}{\partial y} (x - 10)^{2} + (x - y)^{2} + (y - 5)^{2}
\\ = 2(x-y)\times -1 + 2(y-5)\times1
\\= -2x+4y-10$

### Python で偏微分

$(x-10)^2+(x-y)^2+(y-5)^2$

In [10]:
import sympy as sym
sym.init_printing(use_unicode=True)

x ,y  = sym.symbols("x y")
fxy = (x-10)**2+(x-y)**2+(y-5)**2

In [11]:
fdx = sym.diff(fxy,x)
fdx

In [12]:
fdy = sym.diff(fxy,y)
fdy

In [13]:
sym.solve([fdx,fdy])

## 誤差逆伝播法(Backpropagation)

`optimization.ipynb` では、勾配法では、損失関数を重みやバイアスで数値的に微分をして勾配を求めたが、
誤差逆伝播法を使うとより効率よく勾配を求められる。

### 数値的な勾配法

<figure>
    <img src="https://gyazo.com/4a039e7f3554ec91fa35301bd3a88946.png" alt="neural-net">
    <caption><center>2層のニューラルネット</center></caption>
</figure>